In [14]:
import pandas as pd
import csv
import re
import numpy as np 
import matplotlib.pyplot as plt 
#!pip install seaborn
import seaborn as sns
import string
#!pip install nltk
import nltk
import warnings 

from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.tokenize import TweetTokenizer
from nltk.tokenize import TreebankWordTokenizer
nltk.download('stopwords')
nltk.download('punkt')
warnings.filterwarnings("ignore", category=DeprecationWarning)


%matplotlib inline

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Laura\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Laura\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [15]:

df=pd.read_csv("train_tweets.txt",encoding='latin' , names=["target","id","date","Query","user","tweet"])
#print(df['tweet'])
low_memory=False
#df.head(9)
df.shape
print(df['tweet'])


0          @switchfoot http://twitpic.com/2y1zl - Awww, t...
1          is upset that he can't update his Facebook by ...
2          @Kenichan I dived many times for the ball. Man...
3            my whole body feels itchy and like its on fire 
4          @nationwideclass no, it's not behaving at all....
5                              @Kwesidei not the whole crew 
6                                                Need a hug 
7          @LOLTrish hey  long time no see! Yes.. Rains a...
8                       @Tatiana_K nope they didn't have it 
9                                  @twittera que me muera ? 
10               spring break in plain city... it's snowing 
11                                I just re-pierced my ears 
12         @caregiving I couldn't bear to watch it.  And ...
13         @octolinz16 It it counts, idk why I did either...
14         @smarrison i would've been the first, but i di...
15         @iamjazzyfizzle I wish I got to watch it with ...
16         Hollis' death

In [16]:
#Metodo que remueve un patrón usado para quitar las menciones @user
def remove_pattern(input_txt, pattern):
    r = re.findall(pattern, input_txt)
    for i in r:
        input_txt = re.sub(i, '', input_txt)    
    return input_txt 

In [17]:
#Se elminan las menciones del dataset
df.loc[:,'preprocess'] = np.vectorize(remove_pattern)(df['tweet'], "@[\w]*")
#Se remueven las url
df.loc[:,'preprocess'] = df.loc[:,'preprocess'].str.replace("https\S+|http\S+|www.\S+","",case = False)
#Se remueven las apostrofes
df.loc[:,'preprocess'] = df.loc[:,'preprocess'].str.replace("'","")
#Se remueven los caracteres especiales
df.loc[:,'preprocess'] = df.loc[:,'preprocess'].str.replace("[^a-zA-Z#]"," ")
#Convert to lower case
df.loc[:,'preprocess'] = df.loc[:,'preprocess'].str.lower()
#Convert @username to AT_USER
df.loc[:,'preprocess'] = df.loc[:,'preprocess'].str.replace('@[^\s]+', 'AT_USER', regex=True)
#Se elminan todas las palabras de menos de 3 letras
#df.loc[:,'preprocess'] = df.loc[:,'preprocess'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>3]))

In [18]:

#Se usa para visualizar el preprocess del dataset
df.head().loc[:,"tweet":"preprocess"]

,tweet,preprocess
0,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",awww thats a bummer you shoulda got dav...
1,is upset that he can't update his Facebook by ...,is upset that he cant update his facebook by t...
2,@Kenichan I dived many times for the ball. Man...,i dived many times for the ball managed to s...
3,my whole body feels itchy and like its on fire,my whole body feels itchy and like its on fire
4,"@nationwideclass no, it's not behaving at all....",no its not behaving at all im mad why am i...


In [19]:
#Listado de stop_words
stop_words = set(stopwords.words('english')) 
print(stop_words)

{'wasn', "she's", 'won', 'haven', 'from', 'should', 'been', 'the', 'our', 'after', 'under', "shan't", 'she', 'are', 'further', 'isn', "don't", 'weren', 've', 'and', 'needn', 'yourselves', 'himself', 'all', 'hers', 'be', 'where', 'll', 'aren', 'very', "shouldn't", 'against', "couldn't", 'up', 'through', 'than', 'its', 'has', 'didn', "won't", 'theirs', 'this', 'other', 'until', 'about', 'of', 'whom', "hasn't", 'him', 'both', 'shan', 'as', 'now', 'hadn', 'not', 'their', 'only', 't', "doesn't", 'm', 'by', "should've", 'these', 'ain', 'what', 'because', 'your', "didn't", 'who', 'why', "hadn't", 'during', "isn't", 'mightn', 'on', 'in', 'wouldn', 'nor', 'but', 'i', 'her', 'then', 'most', 'ma', "you're", 'or', 'into', 're', 'between', 'each', 'own', 'if', 'few', 'to', 'hasn', 'at', 'it', 'having', 'a', 'will', 'any', 'which', 'itself', 'above', 'his', "it's", 'that', 'yourself', 'me', 's', 'again', 'they', 'ourselves', 'am', 'those', "that'll", 'once', 'being', 'before', "weren't", "mightn't",

In [20]:
#Se convierten las palabras el tweet a elementos de un arreglo
df['word_tokens']=df.loc[:,'preprocess'].apply(TweetTokenizer().tokenize)
print(df['word_tokens'])

0          [awww, thats, a, bummer, you, shoulda, got, da...
1          [is, upset, that, he, cant, update, his, faceb...
2          [i, dived, many, times, for, the, ball, manage...
3          [my, whole, body, feels, itchy, and, like, its...
4          [no, its, not, behaving, at, all, im, mad, why...
5                                    [not, the, whole, crew]
6                                             [need, a, hug]
7          [hey, long, time, no, see, yes, rains, a, bit,...
8                              [nope, they, didnt, have, it]
9                                           [que, me, muera]
10            [spring, break, in, plain, city, its, snowing]
11                          [i, just, re, pierced, my, ears]
12         [i, couldnt, bear, to, watch, it, and, i, thou...
13         [it, it, counts, idk, why, i, did, either, you...
14         [i, wouldve, been, the, first, but, i, didnt, ...
15         [i, wish, i, got, to, watch, it, with, you, i,...
16         [hollis, deat

In [21]:

df['without_stop_words']=df['word_tokens'].apply(lambda x: [i for i in x if i.lower() not in stop_words])
print(df['without_stop_words'])



    
    
 

0          [awww, thats, bummer, shoulda, got, david, car...
1          [upset, cant, update, facebook, texting, might...
2          [dived, many, times, ball, managed, save, rest...
3                    [whole, body, feels, itchy, like, fire]
4                             [behaving, im, mad, cant, see]
5                                              [whole, crew]
6                                                [need, hug]
7          [hey, long, time, see, yes, rains, bit, bit, l...
8                                              [nope, didnt]
9                                               [que, muera]
10                     [spring, break, plain, city, snowing]
11                                           [pierced, ears]
12         [couldnt, bear, watch, thought, ua, loss, emba...
13               [counts, idk, either, never, talk, anymore]
14         [wouldve, first, didnt, gun, really, though, z...
15                        [wish, got, watch, miss, premiere]
16         [hollis, deat

In [22]:
df.head().loc[:,"tweet":"without_stop_words"]

,tweet,preprocess,word_tokens,without_stop_words
0,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",awww thats a bummer you shoulda got dav...,"[awww, thats, a, bummer, you, shoulda, got, da...","[awww, thats, bummer, shoulda, got, david, car..."
1,is upset that he can't update his Facebook by ...,is upset that he cant update his facebook by t...,"[is, upset, that, he, cant, update, his, faceb...","[upset, cant, update, facebook, texting, might..."
2,@Kenichan I dived many times for the ball. Man...,i dived many times for the ball managed to s...,"[i, dived, many, times, for, the, ball, manage...","[dived, many, times, ball, managed, save, rest..."
3,my whole body feels itchy and like its on fire,my whole body feels itchy and like its on fire,"[my, whole, body, feels, itchy, and, like, its...","[whole, body, feels, itchy, like, fire]"
4,"@nationwideclass no, it's not behaving at all....",no its not behaving at all im mad why am i...,"[no, its, not, behaving, at, all, im, mad, why...","[behaving, im, mad, cant, see]"
